## 2nd Group (The ones that we needed to convert from DICOM to NIFTI)
### The folders start with NACC

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import zipfile

In [ ]:
## convert dicom to nifti AND store the protocol names
import os
import subprocess

# the source and the destination directories
source_dir = 'D:\\Babak\\DATA\\NACC\\MRI\\all\\Second_group_dicom_unzipped_shortened'
destination_dir = 'D:\\Babak\\DATA\\NACC\\MRI\\all\\scndgrpconverted'

# to store protocol names
protocol_names = []

# Function to count the total number of sub-subfolders
def count_sub_subfolders(directory):
    count = 0
    for folder_name in os.listdir(directory):
        main_folder = os.path.join(directory, folder_name)
        if os.path.isdir(main_folder):
            for subfolder_name in os.listdir(main_folder):
                subfolder_path = os.path.join(main_folder, subfolder_name)
                if os.path.isdir(subfolder_path):
                    for sub_subfolder_name in os.listdir(subfolder_path):
                        if os.path.isdir(os.path.join(subfolder_path, sub_subfolder_name)):
                            count += 1
    return count

# the total number of sub-subfolders
total_folders = count_sub_subfolders(source_dir)
completed_folders = 0

# iterate over each folder in the source directory
for folder_name in os.listdir(source_dir):
    main_folder = os.path.join(source_dir, folder_name)
    if os.path.isdir(main_folder):
        # Iterate over subfolders in each main folder
        for subfolder_name in os.listdir(main_folder):
            subfolder_path = os.path.join(main_folder, subfolder_name)
            if os.path.isdir(subfolder_path):
                # Iterate over sub-subfolders
                for sub_subfolder_name in os.listdir(subfolder_path):
                    sub_subfolder_path = os.path.join(subfolder_path, sub_subfolder_name)
                    if os.path.isdir(sub_subfolder_path):
                        # Processing starts for each sub-subfolder
                        completed_folders += 1

                        # Create corresponding target folder maintaining the directory structure
                        target_folder = os.path.join(destination_dir, folder_name, subfolder_name, sub_subfolder_name)
                        if not os.path.exists(target_folder):
                            os.makedirs(target_folder)

                        dcm2niix_path = "C:\\Users\\babak.ahmadi\\Downloads\\MRIcron_windows\\MRIcron\\Resources\\dcm2niix"
                        command = f'"{dcm2niix_path}" -o "{target_folder}" -f "%p_%f_%t_%s" -z n -ba y -p y "{sub_subfolder_path}"'

                        # execute the command
                        subprocess.run(command, shell=True, capture_output=True, text=True)

                        # Iterate over NIfTI files in the target folder
                        for file_name in os.listdir(target_folder):
                            if file_name.endswith('.nii') or file_name.endswith('.nii.gz'):
                                # Check the length of the file name
                                if len(file_name) < 30:
                                    # Split the filename at the second "_"
                                    parts = file_name.split('_')
                                    if len(parts) >= 2:
                                        protocol_name = '_'.join(parts[:2])
                                    else:
                                        protocol_name = parts[0]  # Fallback if there aren't enough "_"
                                        print("jaleb")
                                else:
                                    # Extract the first 30 characters
                                    protocol_name = file_name[:30]
                                print(target_folder,":")
                                print(protocol_name)   
                                protocol_names.append(protocol_name)

                        progress_percentage = (completed_folders / total_folders) * 100
                        print(f"Progress: {progress_percentage:.2f}%")

print("Extracted protocol names:", protocol_names)

In [ ]:
import csv

csv_file = r"D:\Babak\DATA\NACC\MRI\all\protocol_names_secondGroup.csv"


# Writing to csv file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Writing each string as a separate row
    for name in protocol_names:
        writer.writerow([name])

print(f"Data written to {csv_file}")

In [ ]:
import pandas as pd
protocols = pd.read_csv(csv_file, names=['protocol_names'])
protocols

In [ ]:
# look at different studies to see how they stored and named their file names
valid_prefixes = [
    "FSPGR_rhuser1", "Sag_3D_T1_FSPGR_BRAVO", "SAG_IRpSPGR_ADNI",
    "SAG_3D", "T1_3D_0.8", "MPRAGE_-_Sag", "T1_C_RF_FAST_VOL_",
    "STRUC_BRAVO_SAG3D_ARC", "MPRAGE_1", "MPRAGE_GRAPPA", 
    "COR_MPRAGE_REFORMAT", "t1sag_208_", "MPRAGE-ADNI", 
    "Accelerated_SAG_IR-SPGR", "MPRAGE_T1_AX", "MPRAGE_Repeat", "MPRAGE_repeat", 
    "SC_STRUC_BRAVO", "3D_Coronal_T1", "SAG_MPRAGE", 
    "Sag_MPRAGE", "T1_3D_SAG_", "SAG_T1_MPRAGE_1_", 
    "SAG_T1_MPRAGE_6_", "SAG_T1_MPRAGE_2_", "SAG_T1_3D_CTSI_T1", 
    "SAG_T1_3D_MPRAGE", "Sag_IR-SPGR", "T1w_MPR_2", "T1w_MPR_1", 
    "MPRAGE_SAG", "MPRAGE_2", "t1_mpr_AX_", "T1_SPGR_1_", 
    "ADNI3_Accelerated_MPRAGE", "Sagittal_T1_Global", 
    "Sagittal_T1w_MPRAGE", "SAGITTAL_T1_VOLUME_SENSE", 
    "SAG_3D_MPRAGE", "MPRAGE_3D_SAG", "MP-RAGE_", 
    "Saggittal_MPRAGE_Siemens_ADNI", "MPRAGE_Siemens_ADNI", 
    "Sagittal_3D_Accelerated_MPRAGE", "t1_fl3d_Coronal", "MPRAGE1x1x1"
]
# Counting the number of protocol names starting with any of the valid prefixes
count = sum(protocols['protocol_names'].apply(lambda name: any(name.startswith(prefix) for prefix in valid_prefixes)))

count

# with that 1439 ==> 4619+1439=6058 out of 6223

In [ ]:
prefix_counts = {prefix: sum(protocols['protocol_names'].str.startswith(prefix)) for prefix in valid_prefixes}
prefix_counts

## 1st Group (The ones that start with a number or are in the form of mri***)

In [ ]:
# 943

import os
import shutil

source_dir = r'D:\Babak\DATA\NACC\MRI\all\nifti'
destination_dir = r'C:\Users\babak.ahmadi\Desktop\NACC_preparation\Patients'

# List all directories in the source directory that start with '943'
for dir_name in os.listdir(source_dir):
    if dir_name.startswith('943') and os.path.isdir(os.path.join(source_dir, dir_name)):
        # Traverse into the first level subdirectory
        first_level_path = os.path.join(source_dir, dir_name)
        for sub_dir in os.listdir(first_level_path):
            second_level_path = os.path.join(first_level_path, sub_dir)

            # Check for 't1' subfolder inside the second level subdirectory
            for sub_sub_dir in os.listdir(second_level_path):
                if sub_sub_dir.lower().startswith('t1'):
                    t1_path = os.path.join(second_level_path, sub_sub_dir)

                    # Look for .nii files inside the 't1' subfolder
                    for file in os.listdir(t1_path):
                        if file.endswith('.nii'):
                            file_path = os.path.join(t1_path, file)

                            # Create the corresponding folder structure in the destination
                            new_folder_path = os.path.join(destination_dir, dir_name)
                            t1_folder_path = os.path.join(new_folder_path, 'T1')
                            os.makedirs(t1_folder_path, exist_ok=True)

                            # Copy the .nii file to the destination folder
                            shutil.copy(file_path, os.path.join(t1_folder_path, file))


In [ ]:
# 1018

# List all directories in the source directory that start with '943'
for dir_name in os.listdir(source_dir):
    if dir_name.startswith('1018') and os.path.isdir(os.path.join(source_dir, dir_name)):
        # Traverse into the first level subdirectory
        first_level_path = os.path.join(source_dir, dir_name)
        for sub_dir in os.listdir(first_level_path):
            second_level_path = os.path.join(first_level_path, sub_dir)

            # Check for 't1' subfolder inside the second level subdirectory
            for sub_sub_dir in os.listdir(second_level_path):
                if sub_sub_dir.startswith('MPRAGE'):
                    t1_path = os.path.join(second_level_path, sub_sub_dir)

                    # Look for .nii files inside the 't1' subfolder
                    for file in os.listdir(t1_path):
                        if file.endswith('.nii'):
                            file_path = os.path.join(t1_path, file)

                            # Create the corresponding folder structure in the destination
                            new_folder_path = os.path.join(destination_dir, dir_name)
                            t1_folder_path = os.path.join(new_folder_path, 'T1')
                            os.makedirs(t1_folder_path, exist_ok=True)

                            # Copy the .nii file to the destination folder
                            shutil.copy(file_path, os.path.join(t1_folder_path, file))

In [ ]:
# 2289

# List all directories in the source directory that start with '943'
for dir_name in os.listdir(source_dir):
    if dir_name.startswith('2289') and os.path.isdir(os.path.join(source_dir, dir_name)):
        # Traverse into the first level subdirectory
        first_level_path = os.path.join(source_dir, dir_name)
        for sub_dir in os.listdir(first_level_path):
            second_level_path = os.path.join(first_level_path, sub_dir)

            # Check for 't1' subfolder inside the second level subdirectory
            for sub_sub_dir in os.listdir(second_level_path):
                if sub_sub_dir.startswith('AcceleratedSagittalIRFSPGR'):
                    t1_path = os.path.join(second_level_path, sub_sub_dir)

                    # Look for .nii files inside the 't1' subfolder
                    for file in os.listdir(t1_path):
                        if file.endswith('.nii'):
                            file_path = os.path.join(t1_path, file)

                            # Create the corresponding folder structure in the destination
                            new_folder_path = os.path.join(destination_dir, dir_name)
                            t1_folder_path = os.path.join(new_folder_path, 'T1')
                            os.makedirs(t1_folder_path, exist_ok=True)

                            # Copy the .nii file to the destination folder
                            shutil.copy(file_path, os.path.join(t1_folder_path, file))

In [ ]:
# 2578

# List all directories in the source directory that start with '943'
for dir_name in os.listdir(source_dir):
    if dir_name.startswith('2578') and os.path.isdir(os.path.join(source_dir, dir_name)):
        # Traverse into the first level subdirectory
        first_level_path = os.path.join(source_dir, dir_name)
        for sub_dir in os.listdir(first_level_path):
            second_level_path = os.path.join(first_level_path, sub_dir)

            # Check for 't1' subfolder inside the second level subdirectory
            for sub_sub_dir in os.listdir(second_level_path):
                if sub_sub_dir.startswith('AcceleratedSagIRFSPGR'):
                    t1_path = os.path.join(second_level_path, sub_sub_dir)

                    # Look for .nii files inside the 't1' subfolder
                    for file in os.listdir(t1_path):
                        if file.endswith('.nii'):
                            file_path = os.path.join(t1_path, file)

                            # Create the corresponding folder structure in the destination
                            new_folder_path = os.path.join(destination_dir, dir_name)
                            t1_folder_path = os.path.join(new_folder_path, 'T1')
                            os.makedirs(t1_folder_path, exist_ok=True)

                            # Copy the .nii file to the destination folder
                            shutil.copy(file_path, os.path.join(t1_folder_path, file))

In [ ]:
# 4967

# List all directories in the source directory that start with '943'
for dir_name in os.listdir(source_dir):
    if dir_name.startswith('4967') and os.path.isdir(os.path.join(source_dir, dir_name)):
        # Traverse into the first level subdirectory
        first_level_path = os.path.join(source_dir, dir_name)
        for sub_dir in os.listdir(first_level_path):
            second_level_path = os.path.join(first_level_path, sub_dir)

            # Check for 't1' subfolder inside the second level subdirectory
            for sub_sub_dir in os.listdir(second_level_path):
                if sub_sub_dir.startswith('T1w'):
                    t1_path = os.path.join(second_level_path, sub_sub_dir)

                    # Look for .nii files inside the 't1' subfolder
                    for file in os.listdir(t1_path):
                        if file.endswith('.nii'):
                            file_path = os.path.join(t1_path, file)

                            # Create the corresponding folder structure in the destination
                            new_folder_path = os.path.join(destination_dir, dir_name)
                            t1_folder_path = os.path.join(new_folder_path, 'T1')
                            os.makedirs(t1_folder_path, exist_ok=True)

                            # Copy the .nii file to the destination folder
                            shutil.copy(file_path, os.path.join(t1_folder_path, file))

In [ ]:
# 8683

# Store the names of folders that were not copied from
folders_not_copied = []

# List all directories in the source directory that start with '943'
for dir_name in os.listdir(source_dir):
    if dir_name.startswith('8683') and os.path.isdir(os.path.join(source_dir, dir_name)):
        copied_from_this_folder = False

        # Traverse into the first level subdirectory
        first_level_path = os.path.join(source_dir, dir_name)
        for sub_dir in os.listdir(first_level_path):
            second_level_path = os.path.join(first_level_path, sub_dir)

            # Check for 'MPrageADNIsag' or 'Sagittal3DMPRAGE' subfolder inside the second level subdirectory
            for sub_sub_dir in os.listdir(second_level_path):
                if sub_sub_dir.startswith('MPrageADNIsag') or sub_sub_dir.startswith('Sagittal3DMPRAGE') or sub_sub_dir.startswith('MPRAGE'):
                    t1_path = os.path.join(second_level_path, sub_sub_dir)

                    # Look for .nii files inside the 'MPrageADNIsag' or 'Sagittal3DMPRAGE' subfolder
                    for file in os.listdir(t1_path):
                        if file.endswith('.nii'):
                            file_path = os.path.join(t1_path, file)

                            # Create the corresponding folder structure in the destination
                            new_folder_path = os.path.join(destination_dir, dir_name)
                            t1_folder_path = os.path.join(new_folder_path, 'T1')
                            os.makedirs(t1_folder_path, exist_ok=True)

                            # Copy the .nii file to the destination folder
                            shutil.copy(file_path, os.path.join(t1_folder_path, file))
                            copied_from_this_folder = True

        # If no file was copied from this folder, add it to the list
        if not copied_from_this_folder:
            folders_not_copied.append(dir_name)

# Print the names of folders that were not copied from
for folder in folders_not_copied:
    print(f"No files were copied from: {folder}")

In [ ]:
# 8734

# Store the names of folders that were not copied from
folders_not_copied = []

# List all directories in the source directory that start with '943'
for dir_name in os.listdir(source_dir):
    if dir_name.startswith('8734') and os.path.isdir(os.path.join(source_dir, dir_name)):
        copied_from_this_folder = False

        # Traverse into the first level subdirectory
        first_level_path = os.path.join(source_dir, dir_name)
        for sub_dir in os.listdir(first_level_path):
            second_level_path = os.path.join(first_level_path, sub_dir)

            # Check for 'MPrageADNIsag' or 'Sagittal3DMPRAGE' subfolder inside the second level subdirectory
            for sub_sub_dir in os.listdir(second_level_path):
                if sub_sub_dir.startswith('3DAxT1MPRAGE'):
                    t1_path = os.path.join(second_level_path, sub_sub_dir)

                    # Look for .nii files inside the 'MPrageADNIsag' or 'Sagittal3DMPRAGE' subfolder
                    for file in os.listdir(t1_path):
                        if file.endswith('.nii'):
                            file_path = os.path.join(t1_path, file)

                            # Create the corresponding folder structure in the destination
                            new_folder_path = os.path.join(destination_dir, dir_name)
                            t1_folder_path = os.path.join(new_folder_path, 'T1')
                            os.makedirs(t1_folder_path, exist_ok=True)

                            # Copy the .nii file to the destination folder
                            shutil.copy(file_path, os.path.join(t1_folder_path, file))
                            copied_from_this_folder = True

        # If no file was copied from this folder, add it to the list
        if not copied_from_this_folder:
            folders_not_copied.append(dir_name)

# Print the names of folders that were not copied from
for folder in folders_not_copied:
    print(f"No files were copied from: {folder}")

In [ ]:
# focus on NACCADC8361 3-level paths
import re

# Store the names of folders that were not copied from and those that were
folders_not_copied = []
folders_copied_from = []

# Counter for the number of folders copied from
copied_folder_count = 0

# List all directories in the source directory that start with 'mri'
for dir_name in os.listdir(source_dir):
    if dir_name.startswith('mri') and os.path.isdir(os.path.join(source_dir, dir_name)):
        copied_from_this_folder = False

        # Traverse into the first level subdirectory (e.g., mri4969ni)
        first_level_path = os.path.join(source_dir, dir_name)

        # Traverse into the second level subdirectory starting with 'NACC'
        for second_dir in os.listdir(first_level_path):
            if second_dir.startswith('NACC'):
                second_level_path = os.path.join(first_level_path, second_dir)

                # Look for a folder that starts with specific strings or patterns
                if os.path.isdir(second_level_path):
                    for third_dir in os.listdir(second_level_path):
                        if (third_dir.startswith('FSPGR_3D_SAG_') or 
                            third_dir.startswith('IRFSPGR_TI=500_FA') or 
                            re.match(r'MPRAGE_\d+', third_dir)):
                            target_path = os.path.join(second_level_path, third_dir)

                            # Traverse into the target folder and look for .nii files
                            if os.path.isdir(target_path):
                                for file in os.listdir(target_path):
                                    if file.endswith('.nii'):
                                        file_path = os.path.join(target_path, file)

                                        # Create the corresponding folder structure in the destination
                                        new_folder_path = os.path.join(destination_dir, dir_name)
                                        t1_folder_path = os.path.join(new_folder_path, 'T1')
                                        os.makedirs(t1_folder_path, exist_ok=True)

                                        # Copy the .nii file to the destination folder
                                        shutil.copy(file_path, os.path.join(t1_folder_path, file))

                                        if not copied_from_this_folder:
                                            copied_from_this_folder = True
                                            copied_folder_count += 1
                                            folders_copied_from.append(dir_name)

        # If no file was copied from this folder, add it to the list
        if not copied_from_this_folder:
            folders_not_copied.append(dir_name)

# Print the names of folders that were not copied from
for folder in folders_not_copied:
    print(f"No files were copied from: {folder}")

# Print the names of folders that were copied from
for folder in folders_copied_from:
    print(f"Copied from: {folder}")

# Print the number of folders files were copied from
print(f"Number of folders files were copied from: {copied_folder_count}")

#1:57 pm 1/8/2024 in NACC_preparation==>Patients

In [ ]:
# focus on NACCADC8361 4-level paths

# Store the names of folders that were not copied from and those that were
folders_not_copied = []
folders_copied_from = []

# Counter for the number of folders copied from
copied_folder_count = 0

# List all directories in the source directory that start with 'mri'
for dir_name in os.listdir(source_dir):
    if dir_name.startswith('mri') and os.path.isdir(os.path.join(source_dir, dir_name)):
        copied_from_this_folder = False

        # Traverse into the first level subdirectory (e.g., mri8112ni)
        first_level_path = os.path.join(source_dir, dir_name)

        # Traverse into the second level subdirectory starting with 'NACC'
        for second_dir in os.listdir(first_level_path):
            if second_dir.startswith('NACC'):
                second_level_path = os.path.join(first_level_path, second_dir)

                # Traverse into the 'nacc' subdirectory
                nacc_path = os.path.join(second_level_path, 'nacc')
                if os.path.isdir(nacc_path):
                    # Look for a folder that starts with specific strings
                    for third_dir in os.listdir(nacc_path):
                        if (third_dir.startswith('FSPGR_SAG_') or 
                            third_dir.startswith('IRSPGR_PROMO_') or 
                            third_dir.startswith('Accelerated_Sag_IRFSPGR_') or 
                            third_dir.startswith('SAG_T1_FSPGR') or 
                            third_dir.startswith('t1_mprage_sag_') or 
                            third_dir.startswith('IR_SPGR_3D_VOLUMETRIC_') or 
                            third_dir.startswith('SAGIRSPGR_TI=500')):
                            target_path = os.path.join(nacc_path, third_dir)

                            # Traverse into the target folder and look for .nii files
                            if os.path.isdir(target_path):
                                for file in os.listdir(target_path):
                                    if file.endswith('.nii'):
                                        file_path = os.path.join(target_path, file)

                                        # Create the corresponding folder structure in the destination
                                        new_folder_path = os.path.join(destination_dir, dir_name)
                                        t1_folder_path = os.path.join(new_folder_path, 'T1')
                                        os.makedirs(t1_folder_path, exist_ok=True)

                                        # Copy the .nii file to the destination folder
                                        shutil.copy(file_path, os.path.join(t1_folder_path, file))

                                        if not copied_from_this_folder:
                                            copied_from_this_folder = True
                                            copied_folder_count += 1
                                            folders_copied_from.append(dir_name)

        # If no file was copied from this folder, add it to the list
        if not copied_from_this_folder:
            folders_not_copied.append(dir_name)

# Print the names of folders that were not copied from
for folder in folders_not_copied:
    print(f"No files were copied from: {folder}")

# Print the names of folders that were copied from
for folder in folders_copied_from:
    print(f"Copied from: {folder}")

# Print the number of folders files were copied from
print(f"Number of folders files were copied from: {copied_folder_count}")


In [ ]:
# just to see how we can check the dimensions of the nifti image and also the orientation and filter based on them


import os
import shutil
import nibabel as nib
import numpy as np

source_dir = r'E:\Babak\DATA\test_nifti6061'
destination_dir = r'C:\Users\babak.ahmadi\Desktop\NACC_preparation\Patients'

folders_not_copied = []
folders_copied_from = []

# Counter for the number of folders copied from
copied_folder_count = 0

# List all directories in the source directory
for dir_name in os.listdir(source_dir):
    if os.path.isdir(os.path.join(source_dir, dir_name)):
        copied_from_this_folder = False

        # Traverse into the subdirectory
        sub_dir_path = os.path.join(source_dir, dir_name)

        # Look for .nii files that end with '_2' or '_3'
        for file in os.listdir(sub_dir_path):
            if file.endswith('.nii') and (file.endswith('_2.nii')): #or file.endswith('_3.nii')):
                file_path = os.path.join(sub_dir_path, file)

                # Load the NIfTI file
                nifti_img = nib.load(file_path)
                img_data = nifti_img.get_fdata()
                header = nifti_img.header

                # Check if the data is 3D
                if img_data.ndim != 3:
                    continue  # Skip non-3D files

                # Get I, J, K dimensions of the image
                i, j, k = img_data.shape

                # Check if the dimensions meet the criteria
                if i < 120 or i > 512 or j < 120 or j > 512 or k < 120 or k > 512:
                    continue  # Skip copying this file

                # Check orientation matrix
                orientation_matrix = header.get_best_affine()[:3, :3]
                if np.array_equal(orientation_matrix, np.array([[1, 0, 0], [0, -1, 0], [0, 0, 1]])):
                    continue  # Skip copying this file

                # Create the corresponding folder structure in the destination
                new_folder_path = os.path.join(destination_dir, dir_name)
                t1_folder_path = os.path.join(new_folder_path, 'T1')
                os.makedirs(t1_folder_path, exist_ok=True)

                # Copy the .nii file to the destination folder
                shutil.copy(file_path, os.path.join(t1_folder_path, file))
                copied_from_this_folder = True

        # Update lists and counters
        if copied_from_this_folder:
            folders_copied_from.append(dir_name)
            copied_folder_count += 1
        else:
            folders_not_copied.append(dir_name)

# Print the names of folders that were not copied from and those that were
for folder in folders_not_copied:
    print(f"No files were copied from: {folder}")

for folder in folders_copied_from:
    print(f"Copied from: {folder}")

# Print the number of folders files were copied from
print(f"Number of folders files were copied from: {copied_folder_count}")
